<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/papers_code/torch_feed_forward_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 2. 데이터 로드/전처리

## 2.1. 데이터 로드

In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((120, 4), (120,), (30, 4), (30,))

In [9]:
print(X_train[:10], y_train[:10])

[[4.6 3.6 1.  0.2]
 [5.7 4.4 1.5 0.4]
 [6.7 3.1 4.4 1.4]
 [4.8 3.4 1.6 0.2]
 [4.4 3.2 1.3 0.2]
 [6.3 2.5 5.  1.9]
 [6.4 3.2 4.5 1.5]
 [5.2 3.5 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.2 4.1 1.5 0.1]] [0 0 1 0 0 2 1 0 0 0]


## 2.2. 스케일링/분할

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 3. 모델링

## 3.1. 모델 정의

- `첫 번째 선형 계층 fully connected layer` iris데이터의 4차원 특성 -> 50차원
- `두 번째 비선형 계층` ReLU 활성화함수로 50차원 -> 50차원
- `세 번째 선형 계층 fully connected layer` 50차원 -> iris데이터의 3개의 클래스
- `네 번째 비선형 계층` Softmax 활성화함수로 3개의 클래스에 대한 확률 계산

In [11]:
class FeedForwardNN(nn.Module):
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

model = FeedForwardNN()

## 3.2. 손실함수/최적화함수

- `CrossEntropyLoss` 다중 클래스 분류 문제에서 자주 사용되는 손실함수
- `Adam(Adaptive Moment Estimation)`은 스토캐스틱 경사 하강법 (SGD) 기반의 최적화 알고리즘 , learning rate는 하이퍼 파라미터(학습률)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 학습

- 전체 데이터셋을 10번 반복
- `model.train()` 모델을 학습모드로 설정
- 현 에폭의 총 손실을 저장할 변수 초기화
- train_loader을 통해 미니배치 단위로 학습데이터 순회

- `optimizer.zero_grad()` 이전 단계에서 계산된 그래디언트 초기화
- `outputs = model(inputs)` 모델에 입력 데이터 통과
- `loss = criterion(outputs, labels)` 예측값과 실제 레이블 간의 손실 계산
- `loss.backward()` 역전파(backpropagation)를 통해 손실의 그래디언트 계산
- `optimizer.step()` 옵티마이저를 사용하여 모델의 파라미터를 업데이트
- `running_loss` 현재 미니배치의 손실 값을 누적

In [16]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/10], Loss: 0.8427
Epoch [2/10], Loss: 0.8256
Epoch [3/10], Loss: 0.8107
Epoch [4/10], Loss: 0.8023
Epoch [5/10], Loss: 0.7984
Epoch [6/10], Loss: 0.7819
Epoch [7/10], Loss: 0.7825
Epoch [8/10], Loss: 0.7731
Epoch [9/10], Loss: 0.7636
Epoch [10/10], Loss: 0.7597


# 5. 평가

- `eval()` 모델을 평가모드로 전환
- `test_loader`에서 미니배치 단위로 데이터 로드
- `outputs = model(inputs)` 모델에 입력 데이터를 통과시켜 예측값을 얻음
- `_, predicted = torch.max(outputs.data, 1)` 출력 값에서 가장 높은 확률을 가진 클래스의 인덱스를 예측값으로 선택. outputs.data는 모델의 출력 텐서입니다.
dim=1은 클래스 차원입니다. 각 샘플에 대해 최대값의 인덱스를 찾습니다.
_는 최대값 자체를 무시한다는 의미, predicted는 최대값의 인덱스를 받습니다. 인덱스는 예측된 클래스 레이블을 나타냅니다.
- `total += labels.size(0)` 총 샘플 수를 누적
- `correct += (predicted == labels).sum().item()` 정확하게 예측한 샘플 수를 누적합니다.

In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')

Accuracy of the model on the test set: 90.00%
